In [70]:
from ultralytics import YOLO
import numpy as np
import os
import cv2
import pytesseract

In [71]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'


In [72]:
#trained yolo model
lp_detect_model = YOLO('yolo_model/best.pt')


In [73]:
#crop the license plate and save as image
def crop_license_plates(img, yolo_model, save_img=True):
    img_name = img.split("\\")[-1]
    
    print(f"Processing {img_name}")
    detected_lps = yolo_model.predict(img)[0]
    license_plates = []
    
    for lp in detected_lps.boxes.data.tolist():
        
        x1, y1, x2, y2, conf, _ = lp
       
        img_array = cv2.imread(img)
        cropped_lp =img_array[int(y1):int(y2), int(x1):int(x2), :]
        license_plates.append(cropped_lp)
        if save_img:
            cv2.imwrite(f"cropped_license_plates/cropped_{img_name}", cropped_lp)
        
        
    return license_plates

In [74]:
def ocr(image):
    custom_config = r'--psm 6 --oem 3 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZÖ0123456789'
    text = pytesseract.image_to_string(image, lang='deu', config=custom_config)
    if text:
        return text.strip("\n")
    return "not detected"

In [75]:
def get_new_w_h(rect):
    (tl, tr, br, bl) = rect
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))

    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))

    maxWidth = max(int(widthA), int(widthB))
    maxHeight = max(int(heightA), int(heightB))
    return maxWidth, maxHeight
    

In [76]:
def warp_image(img, contour):
    
    approx = cv2.approxPolyDP(contour, 0.015 * cv2.arcLength(contour, True), True)
    dst_coords = None
    if len(approx) == 4:
        dst_coords = approx
    if dst_coords is None:
        return None
    points = dst_coords.reshape(4, 2)
    rect = np.zeros((4, 2), dtype = "float32")
    s = points.sum(axis = 1)
    rect[0] = points[np.argmin(s)]
    rect[2] = points[np.argmax(s)]

    diff = np.diff(points, axis = 1)
    rect[1] = points[np.argmin(diff)]
    rect[3] = points[np.argmax(diff)]
    
    maxWidth, maxHeight = get_new_w_h(rect)
    dst = np.array([[0, 0],
                    [maxWidth - 1, 0],
                    [maxWidth - 1, maxHeight - 1],
                    [0, maxHeight - 1]],
                    dtype = "float32")

    M = cv2.getPerspectiveTransform(rect, dst)
    warp = cv2.warpPerspective(img.copy(), M, (maxWidth, maxHeight))
    return warp
    

In [77]:
def apply_image_processing(img, kernel):
    kernel = np.ones((3,3),np.uint8)
    # grayscale
    lp_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # erosion
    lp_gray = cv2.erode(lp_gray, kernel, iterations=1)
    # bilateral filter
    lp_gray = cv2.bilateralFilter(lp_gray, 5, 75, 75)
    # binary thresholding
    _, lp_threshold = cv2.threshold(lp_gray, 6, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    return lp_threshold

In [78]:
def preprocess_license_plate(license_plate, name, save_img=True):
    
    kernel = np.ones((3,3),np.uint8)
    lp_threshold = apply_image_processing(license_plate, kernel)
    
    contours, _ = cv2.findContours(lp_threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if not contours:
        return None
    
    # contour with largest area
    max_contour = max(contours, key=cv2.contourArea)
    #warped_img = warp_image(license_plate.copy(), max_contour)
    
    # Crop the license plate region
    x, y, w, h = cv2.boundingRect(max_contour)
    license_plate_cropped = lp_threshold[y:y + h, x:x + w]

    if save_img:
        cv2.imwrite(f'thresholded_license_plates/thresh_bil_rectified_{name}', license_plate_cropped)

    return license_plate_cropped

In [79]:
#read characters on licenese plate using OCR  
def read_license_plates(license_plates,name, save_img=True):
    kernel = np.ones((3,3),np.uint8)
    for lp in license_plates:

        #pre processing of the image:
        
        #1.    grayscale the image
       
        lp_gray = cv2.cvtColor(lp, cv2.COLOR_BGR2GRAY)

        #2.     erode the image
        lp_gray = cv2.erode(lp_gray, kernel, iterations=1)

        #3a.     remove noise with median filter
        lp_median = cv2.medianBlur(lp_gray,5)
        
        #3b.     remove noise with bilateral filter
        lp_bilateral = cv2.bilateralFilter(lp_gray, 5, 75, 75)

        #4a.    canny edge detection 
        lp_canny = cv2.Canny(lp_median, 100, 200)

        #4b.     binary thresholding 
        _, lp_threshold = cv2.threshold(lp_median, 6, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        _, lp_threshold2 = cv2.threshold(lp_bilateral, 6, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        if save_img:
            cv2.imwrite(f'thresholded_license_plates/thresh_median_{name}', lp_threshold)
            cv2.imwrite(f'thresholded_license_plates/thresh_bil_{name}', lp_threshold2)
            cv2.imwrite(f'thresholded_license_plates/canny_{name}', lp_canny)

        return lp_threshold, lp_threshold2, lp_canny

In [80]:
def find_license_plate_id(img_file):
   
    detected_licenseplates = crop_license_plates(img_file, lp_detect_model)
    assert len(detected_licenseplates) == 1, "WARNING! More than one License plate detected. Access denied!"
    re_cropped = preprocess_license_plate(detected_licenseplates[0], img_file, save_img=False)
    if re_cropped is not None:
        return ocr(re_cropped)
    return None

In [81]:
def main():
    path_to_cam_frames = 'cam_frames'
    all_lp_ids = []
    img_files = [os.path.join(path_to_cam_frames, f) for f in os.listdir(path_to_cam_frames)
                 if f.endswith('.jpg') or f.endswith('.png')]
    all_lp_ids = map(find_license_plate_id, img_files)
    all_lp_ids = [lp_id for lp_id in all_lp_ids if lp_id is not None]
    predicted_license_plate_id = max(set(all_lp_ids), key=all_lp_ids.count)
    print(predicted_license_plate_id)
    

In [83]:
if __name__ == "__main__":
    main()


image 1/1 e:\Users\belau\Documents\BV-ML-CV-Praktikum\cam_frames\image_1.jpg: 512x640 1 license_plate, 148.1ms
Speed: 2.0ms preprocess, 148.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


Processing image_1.jpg



image 1/1 e:\Users\belau\Documents\BV-ML-CV-Praktikum\cam_frames\image_2.jpg: 512x640 1 license_plate, 147.1ms
Speed: 1.0ms preprocess, 147.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)


Processing image_2.jpg



image 1/1 e:\Users\belau\Documents\BV-ML-CV-Praktikum\cam_frames\image_3.jpg: 512x640 1 license_plate, 151.1ms
Speed: 2.0ms preprocess, 151.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


Processing image_3.jpg



image 1/1 e:\Users\belau\Documents\BV-ML-CV-Praktikum\cam_frames\image_4.jpg: 512x640 1 license_plate, 144.1ms
Speed: 2.0ms preprocess, 144.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


Processing image_4.jpg



image 1/1 e:\Users\belau\Documents\BV-ML-CV-Praktikum\cam_frames\image_5.jpg: 512x640 1 license_plate, 144.1ms
Speed: 1.0ms preprocess, 144.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


Processing image_5.jpg
SHGLF2O6


In [82]:
path_to_cam_frames = 'cam_frames'
all_lp_ids = []
for f in os.listdir(path_to_cam_frames):
    if f.endswith('.jpg') or f.endswith('.png'):
        img = os.path.join(path_to_cam_frames, f)

        detected_licenseplates = crop_license_plates(img, lp_detect_model)
        assert len(detected_licenseplates) == 1, "WARNING! More than one License plate detected. Access denied!"
        
        re_cropped = preprocess_license_plate(detected_licenseplates[0], f)
        if re_cropped is not None:
            all_lp_ids.append(ocr(re_cropped))
            
predicted_license_plate_id = max(set(all_lp_ids), key=all_lp_ids.count)   
print(predicted_license_plate_id)      

Processing image_1.jpg


image 1/1 e:\Users\belau\Documents\BV-ML-CV-Praktikum\cam_frames\image_1.jpg: 512x640 1 license_plate, 151.1ms
Speed: 2.0ms preprocess, 151.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 e:\Users\belau\Documents\BV-ML-CV-Praktikum\cam_frames\image_2.jpg: 512x640 1 license_plate, 151.1ms
Speed: 1.0ms preprocess, 151.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


Processing image_2.jpg



image 1/1 e:\Users\belau\Documents\BV-ML-CV-Praktikum\cam_frames\image_3.jpg: 512x640 1 license_plate, 154.1ms
Speed: 1.0ms preprocess, 154.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


Processing image_3.jpg



image 1/1 e:\Users\belau\Documents\BV-ML-CV-Praktikum\cam_frames\image_4.jpg: 512x640 1 license_plate, 157.1ms
Speed: 2.0ms preprocess, 157.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


Processing image_4.jpg



image 1/1 e:\Users\belau\Documents\BV-ML-CV-Praktikum\cam_frames\image_5.jpg: 512x640 1 license_plate, 166.2ms
Speed: 2.0ms preprocess, 166.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


Processing image_5.jpg
SHGLF2O6
